# Продвинутое машинное обучение: ДЗ 2

Второе задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).

<b>Background</b>: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:
построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.


##  Содержание <a name = 'outline'></a>
* [Чтение и анализ данных](#data_analysis)
* [Baseline-модель](#baseline)
* [Оценка качества](#quality)
* [EM алгоритм](#EM)
* [Рейтинг турниров](#rating_tournaments)
* [Рейтинг игроков](#rating_players)

In [151]:
import numpy as np
import pandas as pd
import pickle
import re
import scipy.stats as stats


from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy import sparse as sp
from scipy.special import expit


In [76]:
random_state = 42

## Чтение и анализ данных <a name = "data_analysis"/>

In [88]:
with open('data/results.pkl', 'rb') as r,\
open('data/tournaments.pkl', 'rb') as t,\
open ('data/players.pkl', 'rb') as p:
    results_src = pickle.load(r)
    tournaments_src = pickle.load(t)
    players_src = pickle.load(p)

In [89]:
print('Пример обьекта tournament :')
tournaments_src[1]

Пример обьекта tournament :


{'id': 1,
 'name': 'Чемпионат Южного Кавказа',
 'dateStart': '2003-07-25T00:00:00+04:00',
 'dateEnd': '2003-07-27T00:00:00+04:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/1',
 'orgcommittee': [],
 'synchData': None,
 'questionQty': None}

In [90]:
print('Пример обьекта player :')
players_src[1]

Пример обьекта player :


{'id': 1, 'name': 'Алексей', 'patronymic': None, 'surname': 'Абабилов'}

In [91]:
print('Пример обьекта result для одной из команд:')
results_src[1][0]

Пример обьекта result для одной из команд:


{'team': {'id': 242,
  'name': 'Команда Азимова',
  'town': {'id': 21, 'name': 'Баку'}},
 'mask': None,
 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}},
 'questionsTotal': 0,
 'synchRequest': None,
 'position': 1,
 'controversials': [],
 'flags': [],
 'teamMembers': [{'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 476,
    'name': 'Анар',
    'patronymic': 'Беюкага оглы',
    'surname': 'Азимов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 878,
    'name': 'Фариз',
    'patronymic': 'Наим оглы',
    'surname': 'Аликишибеков'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 1872,
    'name': 'Аднан',
    'patronymic': 'Фариз оглы',
    'surname': 'Ахундов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 13721,
    'name': 'Балаш',
    'patronymic': 'Алекпер оглы',
    'surname': 'Касумов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'play

Отфильтруем турниры, в которых нет повопросных результатов и составов команд

In [105]:
def is_result_clear(team_result):
    return team_result.get('mask') and team_result.get('teamMembers')\
            and len(team_result.get('teamMembers')) > 0\
            and not re.findall('[^01]', team_result.get('mask'))\
            and all(players_src.get(tm['player']['id']) for tm in team_result.get('teamMembers'))

In [106]:
clean_results = {}
clean_tournaments = {}

for t_id, team_results in results_src.items():
    clean_team_results = [tr for tr in team_results if is_result_clear(tr)]
    if (len(clean_team_results) > 0):
        clean_results[t_id] = clean_team_results
        clean_tournaments[t_id] = tournaments_src[t_id]

Разобьем на тестовую и обучающую выборку. В тестовую войдут турниры за 2020 год, а в обучающую - за 2019.

In [107]:
tournaments_train = {k: v for k, v in clean_tournaments.items() if v['dateStart'][:4] == '2019'}
tournaments_test  = {k: v for k, v in clean_tournaments.items() if v['dateStart'][:4] == '2020'}

In [108]:
print(f'Размер обучающей выборки: {len(tournaments_train)}')
print(f'Размер тестовой выборки:  {len(tournaments_test)}')

Размер обучающей выборки: 616
Размер тестовой выборки:  160


## Baseline-модель <a name = "baseline"/>

Посроим baseline-модель на основе логистической регрессии, которая будет обучать рейтинг-лист игроков.

Для того чтобы составить рейтинг лист игроков нужна какая-то мера, по которой можно их сравнить. 

Возьмем вектор OHE-вектор, состоящий из игроков и вопросов. Обучим  логистическую регрессию ответов команд на вопросы. И возьмем за меру, по которой можно сравнивать игроков коэффициент при значении в OHE-векторе соотвествующее определенному игроку.

In [123]:

max_question_id = 0

train_table = []

for tournament_id, tournament in tqdm(tournaments_train.items()):
    for team_result in clean_results[tournament_id]:
        team_id = team_result['team']['id']
        mask = np.array([np.int32(answer) for answer in team_result['mask']])
        players = team_result['teamMembers']
        questions = np.arange(max_question_id, max_question_id + len(mask))
        
        for player in players:
            player_id = player['player']['id']
            for i in range(len(mask)):
                train_table.append([tournament_id, team_id, player_id, questions[i], mask[i]])
    max_question_id += len(mask)    
        
train_df = pd.DataFrame(train_table, 
                        columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'answer'])      

100%|██████████| 616/616 [00:20<00:00, 30.79it/s] 


In [124]:
ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(train_df[['player_id', 'question_id']])
y_train = train_df['answer']

In [125]:
lr = LogisticRegression(random_state=random_state, n_jobs=-1)
lr.fit(X_train, y_train)

LogisticRegression(n_jobs=-1, random_state=42)

In [126]:
active_players = np.unique(train_df['player_id'])
rating = pd.DataFrame({'id': active_players,
                       'coef': lr.coef_[0][:len(active_players)],
                       'name': [(players_src[i]['name'] + ' ' + players_src[i]['surname']) for i in active_players]
                      })

In [127]:
rating.sort_values(by='coef', ascending=False).head(20)

,id,coef,name
3803,27403,3.693135,Максим Руссо
593,4270,3.582439,Александра Брутер
5106,37047,3.440767,Мария Юнгер
5289,38196,3.284078,Артём Митрофанов
3991,28751,3.236148,Иван Семушин
513,3671,3.204605,Алексей Богословский
6633,56647,3.196879,Наталья Горелова
534,3843,3.157081,Светлана Бомешко
4176,30152,3.154473,Артём Сорожкин
3874,27822,3.137638,Михаил Савченков


## Оценка качества <a name = "quality"/>

Отранжировать команды при оценке качества на тестовой выборке можно, исходя из оценки вероятности того, что хотя бы один участник в команде отвечает на вопрос.

In [139]:
test_table = []

for tournament_id, tournament in tqdm(tournaments_test.items()):
    for team_result in clean_results[tournament_id]:
        team_id = team_result['team']['id']
        mask = np.array([np.int32(answer) for answer in team_result['mask']])
        players = team_result['teamMembers']
        
        for player in players:
            player_id = player['player']['id']
            if player_id not in active_players:
                continue
            test_table.append([tournament_id, team_id, player_id, -1, sum(mask), len(mask)])  
        
test_df = pd.DataFrame(test_table, 
                        columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'correct_answers', 'total_answers'])      

100%|██████████| 160/160 [00:03<00:00, 40.93it/s]


In [140]:
X_test = test_df[['player_id', 'question_id']]
X_test = ohe.transform(X_test)
predictions = lr.predict_proba(X_test)[:, 1]

In [175]:
def compute_scores(data, preds):
    data['pred'] = preds
    data['score'] = data.groupby(['tournament_id', 'team_id'])['pred'].transform(lambda x: 1 - np.prod(1 - x))
    rating = data[['tournament_id', 'team_id', 'correct_answers', 'score']].drop_duplicates().reset_index(drop=True)
    
    # Считаем реальный рейтинг команд
    rating = rating.sort_values(by=['tournament_id', 'correct_answers'], ascending=False)
    rating['real_rank'] = rating.groupby('tournament_id')['correct_answers'].transform(lambda x: np.arange(1, len(x) + 1))
    
    # Считаем предсказанный рейтинг
    rating = rating.sort_values(by=['tournament_id', 'score'], ascending=False)
    rating['pred_rank'] = rating.groupby('tournament_id')['score'].transform(lambda x: np.arange(1, len(x) + 1))

    rating = rating.astype(np.int32)
    
    print(f"Корреляция Спирмана: {rating.groupby('tournament_id').apply(lambda x: stats.spearmanr(x['real_rank'], x['pred_rank']).correlation).mean()}")
    print(f"Корреляция Кендалла: {rating.groupby('tournament_id').apply(lambda x: stats.kendalltau(x['real_rank'], x['pred_rank']).correlation).mean()}")

In [142]:
compute_scores(test_df, predictions)

Корреляция Спирмана: 0.7686425235419162
Корреляция Кендалла: 0.6002710452381033


## EM алгоритм <a name = "EM"/>

Усовершенствуем модель. 

Прежде всего необходимо учитывать то, что на вопрос отвечают сразу несколько игроков. В baseline мы считали, что если команда ответила на вопрос, то и каждый игрок в команде ответил на вопрос. Однако в реальности это только дает нам право говорить, что если команда не ответила на вопрос, то ни один игрок в команде на него ответил.
А так же, если хотя бы один игрок ответил на вопрос, то и команда ответила


Введем новые обозначения

Пусть 
$A$ - событие, игрок ответил на вопрос
$B$ - команда ответила на вопрос

Тогда можем вывести следующие соотношения:

$P(B|A) = 1
\\
P(A|\overline{B}) = 0
\\
P(A|B) = \frac{P(B|A)P(A)}{P(B)} = \frac{P(A)}{P(B)}
\\
P(B) = 1 - \prod(1-P(A))
$

Таким образом реализация EM-алгоритма будет выглядеть следующим образом:

E-step - оценка $P(A|B)$

M-step - обучение логистической регрессии на таргете с E-step

В итоге получаем $P(A)$

In [158]:
def log_loss(y_true, y_pred):
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

class EMClassifier:
    
    def __init__(self, w=None, lr=15, n_iter=10, batch_size=5000, verbose=1):
        self.w = w
        self.lr = lr
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.verbose = 1
        
    def _add_intercept(self, X):
        return sp.hstack((np.ones((X.shape[0], 1)), X), format='csr')
    
        
    def _E_step(self, data, preds):
        team_prob = pd.DataFrame({'team_id': data['team_id'],
                                      'question_id': data['question_id'], 
                                      'team_prob': 1 - preds})\
                        .groupby(['team_id', 'question_id']).agg({'team_prob': 'prod'}).reset_index()

        team_prob['team_prob'] = 1 - team_prob['team_prob']
        team_prob = data[['team_id', 'question_id']].merge(team_prob)
        y = np.clip(preds / team_prob['team_prob'], 0, 1).values
        y[data['answer'] == 0] = 0
        return y
        
    def _M_step(self, X, y):
        min_loss = np.inf
        indices = np.arange(X.shape[0])
        for _ in range(100):
            indices = np.random.permutation(indices)
            for batch_idx in np.array_split(indices, len(indices) // self.batch_size):
                x_batch, y_batch = X[batch_idx], y[batch_idx]
                grad = x_batch.T.dot(self.predict(x_batch) - y_batch) / len(y_batch)
                self.w -= self.lr * grad
                
            cur_loss = log_loss(y, self.predict(X))
            if min_loss - cur_loss < 1e-6:
                break
                
            min_loss = cur_loss
                
    def fit(self, X_tr, train_data, X_te=None, test_data=None):
        X_tr = self._add_intercept(X_tr)
        for iter_ in tqdm(range(self.n_iter)): 
            preds = self.predict(X_tr)
            y = self._E_step(train_data, preds)
            self._M_step(X_tr, y)
            if self.verbose is not None and X_te is not None and test_data is not None and iter_ % self.verbose == 0:
                compute_scores(test_data, self.predict(X_te))
                         
    def predict(self, X):
        if len(self.w) != X.shape[1]:
            X = self._add_intercept(X)
        return expit(X.dot(self.w))

In [159]:
# Веса инициализирются предобученной baseline моделью
w_init = np.hstack([lr.intercept_, lr.coef_[0]])
em_classifier = EMClassifier(w_init)

em_classifier.fit(X_train, train_df, X_test, test_df)

 10%|█         | 1/10 [01:53<17:05, 113.95s/it]

Корреляция Спирмана: 0.776488395036184
Корреляция Кендалла: 0.6090761165465001


 20%|██        | 2/10 [07:22<23:45, 178.20s/it]

Корреляция Спирмана: 0.7756330734582625
Корреляция Кендалла: 0.608241723690095


 30%|███       | 3/10 [11:15<22:42, 194.63s/it]

...

Корреляция Спирмана: 0.7773127277069969
Корреляция Кендалла: 0.6092628292823374


 90%|█████████ | 9/10 [42:56<03:57, 237.13s/it]

Корреляция Спирмана: 0.7774699920471461
Корреляция Кендалла: 0.6095793760849213


100%|██████████| 10/10 [46:35<00:00, 279.50s/it]

Корреляция Спирмана: 0.7780886247425316
Корреляция Кендалла: 0.610170353216916


По результатам обучения наблюдается прирост качества, пусть и небольшой.

## Рейтинг турниров <a name = "rating_tournaments"/>

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

Сложность турнира можно посчитать как среднюю сложность вопросов в турнире - возьмем средние коэффициенты обученной модели.

In [164]:
active_questions = np.unique(train_df['question_id'])
q_rating = dict(zip(active_questions, em_classifier.w[-len(active_questions):]))

train_df['difficulty'] = train_df['question_id'].map(q_rating)
train_df['tournament_name'] = train_df['tournament_id'].map({v['id']: v['name']for k, v in tournaments_src.items()})

tournaments_rating = train_df[['tournament_name', 'question_id', 'difficulty']].drop_duplicates()
tournaments_rating = tournaments_rating.groupby('tournament_name')['difficulty'].mean().sort_values().reset_index()

Ниже расположены рейтинг самых сложных турниров

In [165]:
tournaments_rating.head(20)

,tournament_name,difficulty
0,Чемпионат Санкт-Петербурга. Первая лига,-3.593347
1,Угрюмый Ёрш,-2.033586
2,Первенство правого полушария,-1.848786
3,Кубок городов,-1.621134
4,Воображаемый музей,-1.554841
5,Записки охотника,-1.459090
6,Чемпионат России,-1.458811
7,Ускользающая сова,-1.411516
8,All Cats Are Beautiful,-1.409829
9,VERSUS: Коробейников vs. Матвеев,-1.393327


## Рейтинг игроков <a name = "rating_players"/>

Посмотрим еще раз рейтинг игроков

In [169]:
rating = pd.DataFrame({'id': active_players,
                       'coef': em_classifier.w[1:1 + len(active_players)],
                       'name': [(players_src[i]['name'] + ' ' + players_src[i]['surname']) for i in active_players]
                      })
rating['questions_count'] = rating['id'].map(train_df.groupby('player_id')['question_id'].count())

In [174]:
rating.sort_values(by='coef', ascending=False).head(30)

,id,coef,name,questions_count
56810,222188,3.924734,Арина Гринко,216
3803,27403,3.763272,Максим Руссо,2075
9199,87637,3.574147,Антон Саксонов,1179
54137,216863,3.550071,Глеб Гаврилов,252
8063,74001,3.521446,Игорь Мокин,1071
593,4270,3.410755,Александра Брутер,2555
3844,27622,3.391753,Николай Рябых,321
4176,30152,3.296343,Артём Сорожкин,4375
3396,24384,3.271007,Евгений Пашковский,1629
3991,28751,3.270155,Иван Семушин,3386


Среди игроков в топе появилось много игроков, у которых довольно мало сыгранных турниров. Попробуем брать в рейтинг только тех игроков у которых сыграно больше 1000 игр.


In [173]:
rating[rating['questions_count'] > 1000].sort_values(by='coef', ascending=False).head(30)

,id,coef,name,questions_count
3803,27403,3.763272,Максим Руссо,2075
9199,87637,3.574147,Антон Саксонов,1179
8063,74001,3.521446,Игорь Мокин,1071
593,4270,3.410755,Александра Брутер,2555
4176,30152,3.296343,Артём Сорожкин,4375
3396,24384,3.271007,Евгений Пашковский,1629
3991,28751,3.270155,Иван Семушин,3386
4800,34846,3.239409,Антон Чернин,1751
4196,30270,3.203064,Сергей Спешков,3440
3874,27822,3.191496,Михаил Савченков,3107
